Stacked AutoEncoder

In [1]:
# download dataset
###ML-100K


!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

###ML-1M

!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls


--2023-08-06 16:21:20--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  27.5MB/s    in 0.2s    

2023-08-06 16:21:21 (27.5 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In [2]:
# import libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
## Importing the dataset

# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set

training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')


In [4]:
## Getting the number of users and movies

nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))


In [5]:
## Converting the data into an array with users in lines and movies in columns

def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data

In [6]:
training_set = convert(training_set)
test_set = convert(test_set)

In [7]:
## Converting the data into Torch tensors

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [8]:
import torch.nn as nn

class SAE(nn.Module):
    """
    Simple Autoencoder (SAE) Neural Network.

    This class defines the architecture of a simple autoencoder neural network.
    The network consists of four fully connected layers with an activation function
    applied after each hidden layer. The input and output size are determined by
    `nb_movies`.

    Parameters:
        nb_movies (int): Number of input and output features (movies).

    Attributes:
        fc1 (nn.Linear): First fully connected layer with 20 hidden units.
        fc2 (nn.Linear): Second fully connected layer with 10 hidden units.
        fc3 (nn.Linear): Third fully connected layer with 20 hidden units.
        fc4 (nn.Linear): Fourth fully connected layer with `nb_movies` output units.
        activation (nn.Module): Activation function (Sigmoid) applied after each hidden layer.
    """
    def __init__(self, nb_movies):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        """
        Forward pass of the autoencoder network.

        Args:
            x (torch.Tensor): Input tensor to the network.

        Returns:
            torch.Tensor: Output tensor from the network.
        """
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [10]:
sae = SAE(nb_movies)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [11]:
## Training the SAE

nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))


epoch: 1loss: tensor(1.7720)
epoch: 2loss: tensor(1.0967)
epoch: 3loss: tensor(1.0532)
epoch: 4loss: tensor(1.0383)
epoch: 5loss: tensor(1.0308)
epoch: 6loss: tensor(1.0267)
epoch: 7loss: tensor(1.0238)
epoch: 8loss: tensor(1.0219)
epoch: 9loss: tensor(1.0207)
epoch: 10loss: tensor(1.0197)
epoch: 11loss: tensor(1.0186)
epoch: 12loss: tensor(1.0184)
epoch: 13loss: tensor(1.0177)
epoch: 14loss: tensor(1.0174)
epoch: 15loss: tensor(1.0173)
epoch: 16loss: tensor(1.0168)
epoch: 17loss: tensor(1.0166)
epoch: 18loss: tensor(1.0165)
epoch: 19loss: tensor(1.0162)
epoch: 20loss: tensor(1.0162)
epoch: 21loss: tensor(1.0157)
epoch: 22loss: tensor(1.0160)
epoch: 23loss: tensor(1.0158)
epoch: 24loss: tensor(1.0159)
epoch: 25loss: tensor(1.0156)
epoch: 26loss: tensor(1.0157)
epoch: 27loss: tensor(1.0150)
epoch: 28loss: tensor(1.0149)
epoch: 29loss: tensor(1.0130)
epoch: 30loss: tensor(1.0118)
epoch: 31loss: tensor(1.0096)
epoch: 32loss: tensor(1.0092)
epoch: 33loss: tensor(1.0057)
epoch: 34loss: tens

In [12]:
## Testing the SAE

test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9593)
